# Aktywne uczenie
## Przygotowanie środowiska

In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
    print(device)

cuda


In [3]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = True #as True useful with training CNN networks

## Przygotowanie danych - FashionMNIST

In [4]:
transform = transforms.Compose(
    [transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)

In [83]:
# trainset.data = torch.stack([trainset.data]*3, -1)
# testset.data = torch.stack([testset.data]*3, -1)
/temp_loader =torch.utils.data.DataLoader(testset)
next(iter(temp_loader))[0][0][2][35]

tensor([-1.0000, -1.0000, -1.0000, -1.0000, -0.9922, -0.9843, -0.9843, -0.9843,
        -0.9843, -0.9922, -0.9922, -0.9843, -0.9843, -0.9922, -1.0000, -1.0000,
        -1.0000, -0.9922, -0.9922, -0.9843, -0.9843, -0.9765, -0.9059, -0.8196,
        -0.6392, -0.3647, -0.1451, -0.1137, -0.0824, -0.1373, -0.2000, -0.1922,
        -0.1451, -0.1137, -0.0902, -0.0667, -0.0118,  0.0510,  0.1216,  0.1922,
         0.1843,  0.1216,  0.0745,  0.1451,  0.2078,  0.2078,  0.2078,  0.2157,
         0.2157,  0.2314,  0.2392,  0.2471,  0.1843,  0.1216,  0.1216,  0.1451,
         0.1686,  0.2000,  0.2392,  0.3176,  0.3882, -0.1686, -0.7725, -0.9922])

Generate starting indices

In [22]:
def get_train_val():
    rand_sampler = torch.utils.data.RandomSampler(trainset, replacement=False)
    rand_batch_sampler = torch.utils.data.BatchSampler(rand_sampler,
                                                       batch_size=int(0.1*len(trainset)),
                                                       drop_last=False)
    train_idx = []
    for i, batch in enumerate(iter(rand_batch_sampler)):
        if i == 0:
            validation_idx = batch
        else:
            train_idx.extend(batch)
    return train_idx, validation_idx

In [23]:
train_idx_df = pd.DataFrame()
val_idx_df = pd.DataFrame()
for i in range(5):
    train_idx, validation_idx = get_train_val()
    train_idx_df[i] = train_idx
    val_idx_df[i] = validation_idx
train_idx_df.to_csv("train_idx.csv", index=False)
val_idx_df.to_csv("val_idx.csv", index=False)
    

Load starting indices

In [5]:
train_idx_df = pd.read_csv("train_idx.csv")
val_idx_df = pd.read_csv("val_idx.csv")

## Konfiguracja sieci - VGG16

In [6]:
vgg16 = models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT) # get vgg16 model with pretrained weights

# change the number of classes in the last layer
vgg16.classifier[6].out_features = 10
# freeze convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False

input_lastLayer = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(input_lastLayer,10)
vgg16.to(device)
net = vgg16

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [7]:
optimizer = torch.optim.Adam(net.parameters())
loss_module = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

Generate starting state

In [27]:
initial_dict = copy.deepcopy(net.state_dict())
optim_dict = copy.deepcopy(optimizer.state_dict())
sched_dict = copy.deepcopy(scheduler.state_dict())

torch.save(initial_dict, "model_dict.pt")
torch.save(optim_dict, "optimizer_dict.pt")
torch.save(sched_dict, "scheduler_dict.pt")

Load starting state

In [8]:
initial_dict =torch.load("model_dict.pt")
optim_dict = torch.load("optimizer_dict.pt")
sched_dict = torch.load("scheduler_dict.pt")

## Aktywne uczenie - pipeline

In [9]:
def validate(model, dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for int, data in enumerate(dataloader):
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            loss = loss_module(output, target)
            
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
    
    return val_loss, val_accuracy

In [10]:
def active_learn(model, train_data, train_idx, val_idx, heuristic, initial_train_idx, experiment_id):
    model.load_state_dict(initial_dict)
    optimizer.load_state_dict(optim_dict)
    batch_len = int(0.05*len(train_idx)) # calculate length of 5% of training data
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_data, val_idx),
        batch_size=64,
        num_workers=8)
    validation_loss, validation_acc = [], []
    train_loss, train_acc = [], []
    for epoch in range(20):
        print("Epoch: ", epoch)
    
        if epoch == 0:
        # if False:
            # batch_idx, train_idx = generate_random_sample(train_idx, batch_len) # equivalent to first (labeled) dataset
            batch_idx = initial_train_idx
            sub_epochs = 7
        else:
            for group in optimizer.param_groups:
                group['lr'] = 0.001
            old_idx = batch_idx 
            batch_idx, train_idx = generate_heuristic_sample(train_idx, batch_len, model, heuristic, train_data) # equivalent to asking for labelling
            # consider joining the samples
            batch_idx = np.append(old_idx,batch_idx)
            sub_epochs = 7
        print(f"Training on {len(batch_idx)} samples")
        
        # model.load_state_dict(initial_dict)
        # optimizer.load_state_dict(optim_dict)
        # print(model.state_dict(['conv1.weight'])[0])
        model.train()  # turn on training mode
        epoch_subset =  torch.utils.data.Subset(train_data, batch_idx) # get epoch subset 
        epoch_loader =  torch.utils.data.DataLoader(epoch_subset, batch_size=64, num_workers=8, shuffle=True) # convert into loader
        # consider retraining multiple times
        for sub_epoch in range(sub_epochs):
            total = 0   # total n of samples seen
            correct = 0   # total n of coreectly classified samples
            running_loss = 0.0
            for i, data in enumerate(epoch_loader):
                data, target = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()  # zero out the gradients
                output = model(data)  # get the output of the net
                loss = loss_module(output, target)  # calculate the loss 
                running_loss += loss.item()  # add the loss on the subset batch
                _, preds = torch.max(output.data, 1)  # get the predictions
                correct += (preds == target).sum().item() # add the n of correctly classified samples
                total += target.size(0) # add the total of samples seen
                loss.backward()  # backpropagate the weights
                optimizer.step()  # optimize
                del data, target, output, preds
            t_loss = running_loss/total
            t_acc = 100. * correct/total
            val_loss, val_acc = validate(model, val_loader)
            print(f"Sub epoch {sub_epoch} train acc: {t_acc:.2f} train loss: {t_loss:.4f} val acc: {val_acc:.2f} val loss: {val_loss:.4f}")
            scheduler.step(val_loss)
            
        train_loss.append(t_loss)
        train_acc.append(t_acc)

        # val_loss, val_acc = validate(model, val_loader)
        validation_loss.append(val_loss)
        validation_acc.append(val_acc)

        print(f"Train Loss: {t_loss:.4f}, Train Acc: {t_acc:.2f} ",
          f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}")
        if heuristic is not None:
            torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_{heuristic.__name__}.pt')
        else:
            torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_random.pt')
    return train_loss, train_acc, validation_loss, validation_acc

In [11]:
def get_acc_per_class(model, testloader, classes):
    
    model.eval()
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels = data    
            images = images.to(device)
            outputs = model(images).cpu()   
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    all_acc = []
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        all_acc.append(accuracy)
        print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                       accuracy))
    print(f"Average accuracy is {sum(all_acc)/len(all_acc)}")

## Heurystyki

In [12]:
def largest_margin_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        diff = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            batch_diff = torch.max(probs.data, 1)[0] - torch.min(probs.data, 1)[0]
            diff = np.append(diff, batch_diff.cpu().numpy())
        #choose n_samples with smallest ?
        
        smallest = np.argpartition(diff, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [13]:
def smallest_margin_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        diff = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            top2 = torch.topk(probs.data, 2).values
            batch_diff = top2[:,0] - top2[:,1]
            diff = np.append(diff, batch_diff.cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(diff, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [14]:
def least_confidence_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        max_probs = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            max_prob = torch.max(output.data, 1)[0]
            max_probs = np.append(max_probs, max_prob.cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(max_probs, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [15]:
def mc_dropout_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        model.eval()
        count = 0
        for m in model.modules():
            if isinstance(m, nn.Dropout) or isinstance(m, nn.Dropout2d):
                m.train(True)
                count += 1
        assert count > 0, 'We can only do models with dropout!'
        i = 0
        all_results = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            input = data.repeat(7, 1, 1, 1)
            output = model(input).data
            average_output = output.view(7, data.size(0), -1).mean(dim=0)
            probs = torch.softmax(average_output,axis=1)
            entropy = (-probs * probs.log()).sum(dim=1, keepdim=True)
            all_results = np.append(all_results, entropy.cpu().numpy())
            i+=1
        #choose n_samples with largest ?
        # print(all_results, all_results.shape)
        smallest = np.argpartition(all_results, n_samples)[-n_samples:]
        # print(smallest, smallest.shape)
        # print(smallest[:n_samples], smallest[:n_samples].shape)
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [16]:
def score_svm_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    # global svm, train_svm
    svm_results = np.array([])
    # if train_svm:
    if True:
        heuristic_loader = torch.utils.data.DataLoader(
              torch.utils.data.Subset(data, indices),
              batch_size=64,
              num_workers=8
            )
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            svm_result = svm(output)
            svm_results = np.append(svm_results, svm_result.detach().cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(svm_results, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        # train_svm = False
        return chosen, leftover
    else:
        with torch.no_grad():
            heuristic_loader = torch.utils.data.DataLoader(
              torch.utils.data.Subset(data, indices),
              batch_size=64,
              num_workers=8
            )
            for data in heuristic_loader:
                data, target = data[0].to(device), data[1].to(device)
                output = model(data)
                svm_result = svm(output)
                svm_results = np.append(svm_results, svm_result.cpu().numpy())
            #choose n_samples with largest ?
            smallest = np.argpartition(svm_results, n_samples)[-n_samples:]
            chosen = indices[smallest]
            leftover = np.setdiff1d(indices, chosen, assume_unique=True)
            return chosen, leftover

In [17]:
def generate_random_sample(indices, n_samples):
    chosen = np.random.choice(indices, n_samples, replace=False)
    leftover = np.setdiff1d(indices, chosen, assume_unique=True)
  # leftover = np.delete(indices, chosen)
    return chosen, leftover

In [18]:
def generate_heuristic_sample(indices, n_samples, model, heuristic=None, data=None):
    if heuristic is None:
        return generate_random_sample(indices, n_samples)
    else:
        return heuristic(indices, n_samples, model, data)

## Wyniki

## Baseline of 20 epochs on whole dataset

In [22]:
def base_learn(model, train_data, train_idx, val_idx):
    model.load_state_dict(initial_dict)
    optimizer.load_state_dict(optim_dict)
    scheduler.load_state_dict(sched_dict)
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_data, val_idx),
        batch_size=64,
        num_workers=8)
    validation_loss, validation_acc = [], []
    train_loss, train_acc = [], []
    test_loss, test_acc = [], []
    batch_idx = train_idx
    print(f"Training on {len(batch_idx)} samples")
    sub_epochs = 7
    model.train()  # turn on training mode
    epoch_subset =  torch.utils.data.Subset(train_data, batch_idx) # get epoch subset 
    epoch_loader =  torch.utils.data.DataLoader(epoch_subset, batch_size=64, num_workers=8, shuffle=True) # convert into loader
    # consider retraining multiple times
    for sub_epoch in range(sub_epochs):
        total = 0   # total n of samples seen
        correct = 0   # total n of coreectly classified samples
        running_loss = 0.0
        for i, data in enumerate(epoch_loader):
            data, target = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()  # zero out the gradients
            output = model(data)  # get the output of the net
            loss = loss_module(output, target)  # calculate the loss 
            running_loss += loss.item()  # add the loss on the subset batch
            _, preds = torch.max(output.data, 1)  # get the predictions
            correct += (preds == target).sum().item() # add the n of correctly classified samples
            total += target.size(0) # add the total of samples seen
            loss.backward()  # backpropagate the weights
            optimizer.step()  # optimize
            del data, target, output, preds
        t_loss = running_loss/total
        t_acc = 100. * correct/total
        val_loss, val_acc = validate(model, val_loader)
        print(f"Sub epoch {sub_epoch} train acc: {t_acc:.2f} train loss: {t_loss:.4f} val acc: {val_acc:.2f} val loss: {val_loss:.4f}")
        scheduler.step(val_loss)

    train_loss.append(t_loss)
    train_acc.append(t_acc)

    validation_loss.append(val_loss)
    validation_acc.append(val_acc)
    
    ts_loss, ts_acc = validate(model, test_loader)
    test_loss.append(ts_loss)
    test_acc.append(ts_acc)

    print(f"Train Loss: {t_loss:.4f}, Train Acc: {t_acc:.2f} ",
      f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}")
    # if heuristic is not None:
    #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_{heuristic.__name__}.pt')
    # else:
    #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_random.pt')
    return train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc

In [23]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(5):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc = \
        base_learn(net, trainset, train_idx_df[i].to_list(), val_idx_df[i].to_list())
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc

Training on 54000 samples
Sub epoch 0 train acc: 77.92 train loss: 0.0114 val acc: 84.92 val loss: 0.0066
Sub epoch 1 train acc: 87.96 train loss: 0.0053 val acc: 88.70 val loss: 0.0049
Sub epoch 2 train acc: 89.78 train loss: 0.0043 val acc: 89.10 val loss: 0.0051
Sub epoch 3 train acc: 90.75 train loss: 0.0040 val acc: 88.72 val loss: 0.0055
Sub epoch 4 train acc: 91.45 train loss: 0.0036 val acc: 88.77 val loss: 0.0054
Sub epoch 5 train acc: 92.10 train loss: 0.0033 val acc: 89.48 val loss: 0.0049
Sub epoch 6 train acc: 93.01 train loss: 0.0030 val acc: 89.63 val loss: 0.0054
Train Loss: 0.0030, Train Acc: 93.01  Validation Loss: 0.0054, Validation Acc: 89.63
Training on 54000 samples
Sub epoch 0 train acc: 77.73 train loss: 0.0114 val acc: 83.43 val loss: 0.0074
Sub epoch 1 train acc: 87.95 train loss: 0.0053 val acc: 87.98 val loss: 0.0053
Sub epoch 2 train acc: 89.67 train loss: 0.0044 val acc: 88.15 val loss: 0.0053
Sub epoch 3 train acc: 90.64 train loss: 0.0040 val acc: 87.77 

In [25]:
train_loss_df.to_csv("train_loss_fashion_whole.csv")
train_acc_df.to_csv("train_acc_fashion_whole.csv")
val_loss_df.to_csv("val_loss_fashion_whole.csv")
val_acc_df.to_csv("val_acc_fashion_whole.csv")
test_loss_df.to_csv("test_loss_fashion_whole.csv")
test_acc_df.to_csv("test_acc_fashion_whole.csv")

Generate first batch

In [22]:
batch_df = pd.DataFrame()
train_df = pd.DataFrame()
for i in range(5):
    i = str(i)
    train_idx = train_idx_df[i].to_list()
    batch_idx, train_idx = generate_random_sample(train_idx, 4*2250)
    batch_df[i] = batch_idx
    train_df[i] = train_idx
batch_df.to_csv("first_batch_idx.csv", index=False)
train_df.to_csv("other_train_idx.csv", index=False)

Load first batch

In [19]:
batch_df = pd.read_csv("first_batch_idx.csv")
train_df = pd.read_csv("other_train_idx.csv")

### Wybór losowy

In [23]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, train_df[i].to_list(), val_idx_df[i].to_list(), 
                     heuristic=None, initial_train_idx=batch_df[i].to_list(),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    


Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 72.57 train loss: 0.0142 val acc: 81.00 val loss: 0.0081
Sub epoch 1 train acc: 84.49 train loss: 0.0070 val acc: 85.27 val loss: 0.0066
Sub epoch 2 train acc: 88.83 train loss: 0.0049 val acc: 85.70 val loss: 0.0066
Sub epoch 3 train acc: 91.09 train loss: 0.0038 val acc: 84.32 val loss: 0.0079
Sub epoch 4 train acc: 91.09 train loss: 0.0039 val acc: 86.37 val loss: 0.0072
Sub epoch 5 train acc: 92.92 train loss: 0.0031 val acc: 85.90 val loss: 0.0078
Sub epoch 6 train acc: 93.83 train loss: 0.0028 val acc: 86.07 val loss: 0.0085
Train Loss: 0.0028, Train Acc: 93.83  Validation Loss: 0.0085, Validation Acc: 86.07
Epoch:  1
Training on 11250 samples
Sub epoch 0 train acc: 81.04 train loss: 0.0141 val acc: 82.40 val loss: 0.0098
Sub epoch 1 train acc: 90.75 train loss: 0.0042 val acc: 87.52 val loss: 0.0066
Sub epoch 2 train acc: 94.98 train loss: 0.0022 val acc: 87.77 val loss: 0.0077
Sub epoch 3 train acc: 96.41 train loss: 0.0

In [24]:
train_loss_df.to_csv("t_loss_random.csv",index=False)
train_acc_df.to_csv("t_acc_random.csv",index=False)
val_loss_df.to_csv("v_loss_random.csv",index=False)
val_acc_df.to_csv("v_acc_random.csv",index=False)

In [25]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_random.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

85.44
86.97
86.48
87.48
86.18
88.13
87.6
88.39
87.86
87.64
88.59
87.8
89.09
88.26
88.97
88.7
89.49
89.16
88.95
89.49
[0.009525631396472454, 0.010316131988167762, 0.013447535765171052, 0.009738220472633839, 0.013471759486198425, 0.012005363115295768, 0.012922690609097481, 0.011377813646942377, 0.012481050392985344, 0.013728470582515001, 0.012797351814806462, 0.014021091724187135, 0.010831553285755217, 0.012995947771705686, 0.01398697234094143, 0.013313394506275654, 0.011538404696807265, 0.01488361993022263, 0.015001903466880321, 0.013418179763108492] [85.44, 86.97, 86.48, 87.48, 86.18, 88.13, 87.6, 88.39, 87.86, 87.64, 88.59, 87.8, 89.09, 88.26, 88.97, 88.7, 89.49, 89.16, 88.95, 89.49]
85.9
86.41
87.66
86.63
87.42
87.67
88.38
87.93
87.98
89.11
88.5
88.78
89.0
88.19
88.82
88.86
88.89
89.28
88.85
88.58
[0.009107118813693523, 0.011975440914928913, 0.010064474602788687, 0.010730746471881866, 0.011462384861707687, 0.012580435878783464, 0.010085494581609964, 0.011069421184062957, 0.0120491690

In [26]:
val_loss_df.to_csv("val_loss_random.csv",index=False)
val_acc_df.to_csv("val_acc_random.csv",index=False)
test_loss_df.to_csv("test_loss_random.csv",index=False)
test_acc_df.to_csv("test_acc_random.csv",index=False)

### Najmniejsza pewność

In [27]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=least_confidence_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 72.59 train loss: 0.0141 val acc: 81.75 val loss: 0.0082
Sub epoch 1 train acc: 84.54 train loss: 0.0068 val acc: 83.28 val loss: 0.0083
Sub epoch 2 train acc: 88.42 train loss: 0.0051 val acc: 85.12 val loss: 0.0068
Sub epoch 3 train acc: 90.83 train loss: 0.0040 val acc: 86.22 val loss: 0.0070
Sub epoch 4 train acc: 91.83 train loss: 0.0035 val acc: 86.42 val loss: 0.0072
Sub epoch 5 train acc: 92.90 train loss: 0.0032 val acc: 86.22 val loss: 0.0084
Sub epoch 6 train acc: 93.57 train loss: 0.0028 val acc: 84.82 val loss: 0.0102
Train Loss: 0.0028, Train Acc: 93.57  Validation Loss: 0.0102, Validation Acc: 84.82
Epoch:  1
Training on 11250 samples
Sub epoch 0 train acc: 73.39 train loss: 0.0169 val acc: 84.05 val loss: 0.0074
Sub epoch 1 train acc: 86.44 train loss: 0.0061 val acc: 87.75 val loss: 0.0061
Sub epoch 2 train acc: 91.19 train loss: 0.0037 val acc: 87.83 val loss: 0.0080
Sub epoch 3 train acc: 93.05 train loss: 0.0

In [28]:
train_loss_df.to_csv("t_loss_lc.csv",index=False)
train_acc_df.to_csv("t_acc_lc.csv",index=False)
val_loss_df.to_csv("v_loss_lc.csv",index=False)
val_acc_df.to_csv("v_acc_lc.csv",index=False)

In [29]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_least_confidence_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

83.82
86.28
87.92
87.96
89.1
88.83
89.4
88.8
88.48
89.66
88.81
89.43
89.48
89.02
89.66
88.77
88.68
89.74
88.94
89.54
[0.010846050864458084, 0.012131248204410076, 0.010629665391519666, 0.012602807514369489, 0.010652915196120738, 0.012366693031042814, 0.0117666459152475, 0.012304335830360651, 0.01149319866001606, 0.011627317145839334, 0.01211261975876987, 0.010741387387178837, 0.01207820691820234, 0.012832903622835875, 0.01526280838958919, 0.01652849762365222, 0.016938083976507187, 0.01729756361544132, 0.016354300340265037, 0.01668296679109335] [83.82, 86.28, 87.92, 87.96, 89.1, 88.83, 89.4, 88.8, 88.48, 89.66, 88.81, 89.43, 89.48, 89.02, 89.66, 88.77, 88.68, 89.74, 88.94, 89.54]
86.81
85.81
88.42
88.97
88.35
89.09
88.75
88.54
89.47
89.29
89.73
89.4
88.57
89.52
89.4
88.92
89.48
89.04
89.23
89.6
[0.008059917865693569, 0.01118792281150818, 0.009270594184845686, 0.010665647437423468, 0.01136741586253047, 0.011137138981372119, 0.010491987311095, 0.01214660474807024, 0.011603502813726664, 0.0

In [30]:
val_loss_df.to_csv("val_loss_lc.csv",index=False)
val_acc_df.to_csv("val_acc_lc.csv",index=False)
test_loss_df.to_csv("test_loss_lc.csv",index=False)
test_acc_df.to_csv("test_acc_lc.csv",index=False)

### Najmniejszy margines

In [20]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=smallest_margin_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 72.51 train loss: 0.0139 val acc: 81.77 val loss: 0.0091
Sub epoch 1 train acc: 84.57 train loss: 0.0069 val acc: 85.93 val loss: 0.0066
Sub epoch 2 train acc: 88.08 train loss: 0.0051 val acc: 86.73 val loss: 0.0061
Sub epoch 3 train acc: 90.81 train loss: 0.0040 val acc: 84.72 val loss: 0.0071
Sub epoch 4 train acc: 92.00 train loss: 0.0035 val acc: 85.12 val loss: 0.0090
Sub epoch 5 train acc: 93.08 train loss: 0.0031 val acc: 84.83 val loss: 0.0095
Sub epoch 6 train acc: 93.38 train loss: 0.0030 val acc: 87.03 val loss: 0.0077
Train Loss: 0.0030, Train Acc: 93.38  Validation Loss: 0.0077, Validation Acc: 87.03
Epoch:  1
Training on 11250 samples
Sub epoch 0 train acc: 72.36 train loss: 0.0168 val acc: 85.85 val loss: 0.0075
Sub epoch 1 train acc: 85.47 train loss: 0.0062 val acc: 87.27 val loss: 0.0066
Sub epoch 2 train acc: 89.62 train loss: 0.0041 val acc: 88.17 val loss: 0.0076
Sub epoch 3 train acc: 91.80 train loss: 0.0

In [21]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_smallest_margin_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.00850279885828495, 0.01094275106266141, 0.010985550126433373, 0.010955968340486288, 0.010293313064798713, 0.01146764251217246, 0.0117316101802513, 0.01210487645715475, 0.009209453854709863, 0.011155574430525303, 0.012956572581082583, 0.01312733051404357, 0.012662731570005416, 0.013357287845015525, 0.015396043821424246, 0.015175096303969621, 0.015565117588639259, 0.014985355918109417, 0.017139974722266197, 0.01664181820973754] [85.82, 86.68, 88.04, 88.93, 88.77, 88.45, 89.2, 88.71, 89.44, 88.7, 89.42, 88.56, 89.5, 89.29, 88.57, 89.4, 88.42, 88.56, 89.5, 88.68]
[0.00670951359719038, 0.007821355031430721, 0.010095248490571977, 0.0084762027669698, 0.010289756299555301, 0.011323608185350895, 0.011802571365237236, 0.010857636739313602, 0.010739322362840175, 0.01390401451215148, 0.012665992404520512, 0.01206184198986739, 0.011357007205486297, 0.013619412818085402, 0.014408696500211954, 0.013848421279340982, 0.014816562727093696, 0.0164042951811105, 0.016008747147768735, 0.01745810698717832

In [22]:
val_loss_df.to_csv("val_loss_sm.csv",index=False)
val_acc_df.to_csv("val_acc_sm.csv",index=False)
test_loss_df.to_csv("test_loss_sm.csv",index=False)
test_acc_df.to_csv("test_acc_sm.csv",index=False)

### Największy margines

In [23]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=largest_margin_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 72.67 train loss: 0.0142 val acc: 82.57 val loss: 0.0079
Sub epoch 1 train acc: 85.03 train loss: 0.0068 val acc: 86.78 val loss: 0.0058
Sub epoch 2 train acc: 89.06 train loss: 0.0049 val acc: 84.73 val loss: 0.0068
Sub epoch 3 train acc: 93.54 train loss: 0.0028 val acc: 88.23 val loss: 0.0057
Sub epoch 4 train acc: 95.50 train loss: 0.0020 val acc: 88.18 val loss: 0.0058
Sub epoch 5 train acc: 96.57 train loss: 0.0016 val acc: 87.80 val loss: 0.0061
Sub epoch 6 train acc: 97.41 train loss: 0.0012 val acc: 88.32 val loss: 0.0065
Train Loss: 0.0012, Train Acc: 97.41  Validation Loss: 0.0065, Validation Acc: 88.32
Epoch:  1
Training on 11250 samples
Sub epoch 0 train acc: 66.57 train loss: 0.0209 val acc: 82.28 val loss: 0.0081
Sub epoch 1 train acc: 79.72 train loss: 0.0082 val acc: 88.00 val loss: 0.0056
Sub epoch 2 train acc: 85.07 train loss: 0.0059 val acc: 88.22 val loss: 0.0057
Sub epoch 3 train acc: 88.60 train loss: 0.0

In [24]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_largest_margin_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.007315501609444618, 0.009128691370785237, 0.010758210846409202, 0.010151913174614311, 0.011240235884860159, 0.008678121334686876, 0.009817339851334692, 0.01062840114980936, 0.010194336104393005, 0.011200559078156948, 0.01257193700298667, 0.013291245955228806, 0.01333250345736742, 0.013487906490638852, 0.015669973218347878, 0.015749315512925386, 0.012892290412448347, 0.015163356708362698, 0.015294528330117465, 0.01626269641183317] [87.23, 87.28, 88.1, 89.19, 88.72, 89.36, 89.37, 88.61, 89.45, 89.58, 88.98, 89.28, 89.12, 89.05, 89.28, 89.21, 89.72, 89.45, 88.83, 89.48]
[0.008011613088846207, 0.011610482408106327, 0.009774842622876168, 0.010948193832486868, 0.012980619324743748, 0.011376658616960049, 0.011853212324902416, 0.00837470477372408, 0.01105210838392377, 0.013304176506400109, 0.010909470396488905, 0.011708913784474135, 0.013817470186203717, 0.014680499859899282, 0.01591632227152586, 0.01582236551195383, 0.01710845746099949, 0.018078218510653825, 0.016278653800487517, 0.0185229

In [25]:
val_loss_df.to_csv("val_loss_lm.csv",index=False)
val_acc_df.to_csv("val_acc_lm.csv",index=False)
test_loss_df.to_csv("test_loss_lm.csv",index=False)
test_acc_df.to_csv("test_acc_lm.csv",index=False)

### Monte Carlo

In [26]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=mc_dropout_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 72.93 train loss: 0.0138 val acc: 77.88 val loss: 0.0102
Sub epoch 1 train acc: 86.56 train loss: 0.0060 val acc: 86.78 val loss: 0.0059
Sub epoch 2 train acc: 89.49 train loss: 0.0045 val acc: 87.53 val loss: 0.0056
Sub epoch 3 train acc: 91.40 train loss: 0.0038 val acc: 88.15 val loss: 0.0054
Sub epoch 4 train acc: 92.53 train loss: 0.0031 val acc: 88.03 val loss: 0.0054
Sub epoch 5 train acc: 94.34 train loss: 0.0025 val acc: 88.12 val loss: 0.0055
Sub epoch 6 train acc: 95.63 train loss: 0.0020 val acc: 88.05 val loss: 0.0060
Train Loss: 0.0020, Train Acc: 95.63  Validation Loss: 0.0060, Validation Acc: 88.05
Epoch:  1
Training on 11250 samples
Sub epoch 0 train acc: 71.68 train loss: 0.0210 val acc: 80.95 val loss: 0.0104
Sub epoch 1 train acc: 85.14 train loss: 0.0070 val acc: 86.53 val loss: 0.0061
Sub epoch 2 train acc: 89.72 train loss: 0.0044 val acc: 86.97 val loss: 0.0061
Sub epoch 3 train acc: 91.97 train loss: 0.0

In [27]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_mc_dropout_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.006454977619647979, 0.007513852410018444, 0.012177833758294582, 0.009188270650804043, 0.011750051452219487, 0.01471312368772924, 0.010758466263860464, 0.011852485623955727, 0.01208242296129465, 0.01321757923066616, 0.01455217830836773, 0.00997222841978073, 0.011704740974283777, 0.012340587155148387, 0.010681984854862095, 0.012840399500261992, 0.012965312905609608, 0.012117489473894238, 0.015186554259806872, 0.01399996635504067] [86.89, 87.41, 86.01, 87.81, 87.59, 87.63, 88.26, 86.99, 87.58, 88.32, 87.92, 88.56, 88.6, 88.08, 89.0, 88.47, 88.29, 88.61, 88.1, 88.74]
[0.007729027733206749, 0.009311564723402261, 0.007847379719465971, 0.008973601446300746, 0.011111704054474831, 0.01057498421818018, 0.011665396835654974, 0.01127609640955925, 0.011093707536906004, 0.012942299670353532, 0.015303342847526073, 0.012917837099730968, 0.012998149076104163, 0.010017135564237833, 0.01199440236017108, 0.014525871756672859, 0.01114669477045536, 0.014042772030830384, 0.0168311753064394, 0.013076809017

In [28]:
val_loss_df.to_csv("val_loss_mc.csv",index=False)
val_acc_df.to_csv("val_acc_mc.csv",index=False)
test_loss_df.to_csv("test_loss_mc.csv",index=False)
test_acc_df.to_csv("test_acc_mc.csv",index=False)

### Maszyna wektorów nośnych

In [ ]:
# train_loss_df = pd.DataFrame()
# train_acc_df = pd.DataFrame()
# val_loss_df = pd.DataFrame()
# val_acc_df = pd.DataFrame()
# for i in range(3):
#     i = str(i)
#     train_loss, train_acc, validation_loss, validation_acc = \
#         active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
#                       heuristic=mc_dropout_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
#                      experiment_id=i)
#     train_loss_df[i] = train_loss
#     train_acc_df[i] = train_acc
#     val_loss_df[i] = validation_loss
#     val_acc_df[i] = validation_acc

In [ ]:
# val_loss_df = pd.DataFrame()
# val_acc_df = pd.DataFrame()
# test_loss_df = pd.DataFrame()
# test_acc_df = pd.DataFrame()
# test_loader = torch.utils.data.DataLoader(
#         testset,
#         batch_size=64,
#         num_workers=8)
# for i in range(3):
#     i = str(i)
#     validation_loss, validation_acc = [], []
#     test_loss, test_acc = [], []
#     val_loader = torch.utils.data.DataLoader(
#         torch.utils.data.Subset(trainset, val_idx_df[i]),
#         batch_size=64,
#         num_workers=8)
#     for k in range(20):
#         net.load_state_dict(torch.load(f"{i}/epoch{k}_mc_dropout_heuristic.pt"))
        
#         v_loss, v_acc = validate(net, val_loader)
#         t_loss, t_acc = validate(net, test_loader)
#         validation_loss.append(v_loss)
#         validation_acc.append(v_acc)
#         test_loss.append(t_loss)
#         test_acc.append(t_acc)
#         # print(t_acc)

#     test_loss_df[i] = test_loss
#     test_acc_df[i] = test_acc
#     val_loss_df[i] = validation_loss
#     val_acc_df[i] = validation_acc
#     print(test_loss, test_acc)

In [ ]:
# val_loss_df.to_csv("val_loss_mc.csv",index=False)
# val_acc_df.to_csv("val_acc_mc.csv",index=False)
# test_loss_df.to_csv("test_loss_mc.csv",index=False)
# test_acc_df.to_csv("test_acc_mc.csv",index=False)